In [69]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dense, InputLayer, Embedding, Flatten
from tensorflow.keras import Sequential
from tensorflow.keras.datasets import imdb
import numpy as np

In [5]:
num_distinct_words = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_distinct_words)

In [7]:
x_train.shape, y_train.shape

((25000,), (25000,))

In [11]:
len(x_train[0]), len(x_train[1])

(218, 189)

In [12]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64)

In [13]:
lens = [len(i) for i in x_train]

In [14]:
max(lens)

2494

#### pad_sequences

In [15]:
sequence = [[1], [2, 3], [4, 5, 6]]
tf.keras.preprocessing.sequence.pad_sequences(sequence)

array([[0, 0, 1],
       [0, 2, 3],
       [4, 5, 6]])

In [16]:
sequence = [[1], [2, 3], [4, 5, 6]]
tf.keras.preprocessing.sequence.pad_sequences(sequence, value=-1)

array([[-1, -1,  1],
       [-1,  2,  3],
       [ 4,  5,  6]])

In [17]:
sequence = [[1], [2, 3], [4, 5, 6]]
tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=5, value=-2)

array([[-2, -2, -2, -2,  1],
       [-2, -2, -2,  2,  3],
       [-2, -2,  4,  5,  6]])

In [18]:
sequence = [[1], [2, 3], [4, 5, 6]]
tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=2, value=-2)

array([[-2,  1],
       [ 2,  3],
       [ 5,  6]])

#### pad the x_train and x_test

In [19]:
x_train_padded = pad_sequences(x_train, maxlen=1000, value=0)
x_test_padded = pad_sequences(x_test, maxlen=1000, value=0)


In [22]:
model = Sequential([InputLayer(input_shape=(1000, )),
                   Embedding(input_dim=num_distinct_words, output_dim=16),
                   LSTM(16),
                   Dense(1, activation='sigmoid')])

In [26]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')

In [27]:
model.fit(x_train_padded, y_train, epochs=5, verbose=1)

Epoch 1/5
782/782 [==============================] - 221s 281ms/step - loss: 0.4135 - acc: 0.8279
Epoch 2/5
782/782 [==============================] - 227s 291ms/step - loss: 0.2685 - acc: 0.8963
Epoch 3/5
782/782 [==============================] - 228s 291ms/step - loss: 0.2302 - acc: 0.9133
Epoch 4/5
782/782 [==============================] - 228s 292ms/step - loss: 0.2027 - acc: 0.9232
Epoch 5/5
782/782 [==============================] - 227s 290ms/step - loss: 0.4020 - acc: 0.8136


In [28]:
model = Sequential([
                   Embedding(input_dim=num_distinct_words, output_dim=16, input_length=1000), #output will be (batch_size, 1000, 16)
                   LSTM(16, return_sequences=False), 
                   Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
model.fit(x_train_padded, y_train, epochs=3, verbose=1)

Epoch 1/3
782/782 [==============================] - 210s 266ms/step - loss: 0.4221 - acc: 0.8009
Epoch 2/3
782/782 [==============================] - 226s 289ms/step - loss: 0.2861 - acc: 0.8880
Epoch 3/3
782/782 [==============================] - 227s 290ms/step - loss: 0.2379 - acc: 0.9078


In [70]:
model = Sequential([
                   Embedding(input_dim=num_distinct_words, output_dim=16, input_length=1000), #output will be (batch_size, 1000, 16)
                   LSTM(16, return_sequences=True), # This will return shape: (Batch_size, 1000, 16)
                    Flatten(),
                   Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
model.fit(x_train_padded, y_train, epochs=3, verbose=1)

Epoch 1/3
782/782 [==============================] - 211s 268ms/step - loss: 0.3671 - acc: 0.8277
Epoch 2/3
782/782 [==============================] - 226s 289ms/step - loss: 0.2343 - acc: 0.9065
Epoch 3/3
782/782 [==============================] - 227s 291ms/step - loss: 0.1894 - acc: 0.9256


In [62]:
x = np.random.randint(low=0, high=10, size=(3, 4))
x

array([[1, 9, 2, 3],
       [5, 1, 7, 6],
       [8, 4, 5, 6]])

In [63]:
x_embed = Embedding(input_dim=10, output_dim=5, input_length=4)(x) # (batch_size, 4, 5)
print(x_embed.shape)
lstm_output = LSTM(6, return_sequences=False)(x_embed)
print(lstm_output.shape)

(3, 4, 5)
(3, 6)


In [67]:
x = np.random.randint(low=0, high=10, size=(3, 4))
x_embed = Embedding(input_dim=10, output_dim=5, input_length=4)(x) # (batch_size, 4, 5)
print(x_embed.shape)
lstm_output = LSTM(6, return_sequences=True)(x_embed)
print(lstm_output.shape)

(3, 4, 5)
(3, 4, 6)
